In [0]:
!pip install Keras==2.2.4

In [0]:
!pip install tensorflow==1.14.0rc1

In [0]:
import pandas as pd
import numpy as np

from numpy.random import seed
from tensorflow import set_random_seed
import tensorflow as tf
import random as rn
import os
from nltk import tokenize 
import nltk
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
#2. Get the file
if os.name=='posix':
    
    # GOOGLE COLAB SETUP
    # Load the Drive helper and mount
    from google.colab import drive
    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
    codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/AOBDL_DL'
    cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models/cv_models_twitter'
    models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models/models_twitter'
    
else:
    data_path          = '../data'
    codes_path         = '../AOBDL_DL'
    cv_models_path     = '../cv_models/cv_models_twitter'
    models_path        = '../models/models_twitter'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append(f'{codes_path}')

In [0]:
if tf.test.is_gpu_available():
  import models
else:
  import models_no_gpu as models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/twitter_cleaned_no_punkt.csv') 
train.drop(['count', 'hate_speech', 'offensive_language', 'neither', 'class'], axis=1, inplace=True)
train.tweet.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['tweet', 'mal'])

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42      
X, X_test, y, y_test  = train_test_split(train.tweet, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
max_features = 10000
maxlen       = 33
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150
threshold    = 0.7


max_sen_len    = 100
max_sent_amount = 4

seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
length = pd.Series(X).apply(lambda x: len(x.split()))

In [0]:
length_sen = pd.Series(X).apply(lambda x: len(x.split('.|!|?')))

In [0]:
length.describe()

count    17549.000000
mean        12.800387
std          6.045859
min          1.000000
25%          8.000000
50%         12.000000
75%         18.000000
max         33.000000
dtype: float64

# LSTM

In [0]:
models.train_model(
    X=X, 
    y=y,
    mtype = 'LSTM',
    cv=True, 
    nfolds=5, 
    epochs=4, 
    cv_models_path=cv_models_path, 
    train=False, 
    rs=rs, 
    max_features=max_features, 
    maxlen=maxlen,
    threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Fitting
Epoch 1/4
14039/14039 [==============================] - 16s 1ms/step - loss: 0.4100 - acc: 0.8316
Epoch 2/4
14039/14039 [==============================] - 15s 1ms/step - loss: 0.1917 - acc: 0.9221
Epoch 3/4
14039/14039 [==============================] - 15s 1ms/step - loss: 0.1134 - acc: 0.9585
Epoch 4/4
3510/3510 [==============================] - 1s 313us/step
 0.7 fold 0 precision 0.984 recall 0.957 fscore 0.97
fold 0 average precision 0.996
fold 0 roc auc 0.982
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 16s 1ms/step - loss: 0.4371 - acc: 0.8250
Epoch 2/4
14039/14039 [==============================] - 15s 1ms/step - loss: 0.2318 - acc: 0.9041
Epoch 3/4
14039/14039 [==============================] - 15s 1ms/step - loss: 0.1188

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'LSTM', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 20s 1ms/step - loss: 0.3959 - acc: 0.8390
Epoch 2/4
17549/17549 [==============================] - 19s 1ms/step - loss: 0.1711 - acc: 0.9346
Epoch 3/4
17549/17549 [==============================] - 19s 1ms/step - loss: 0.1030 - acc: 0.9631
Epoch 4/4
7168/7168 [==============================] - 2s 297us/step
_________________________________
PR-C is 0.996
_________________________________

_________________________________
ROC AUC is 0.981
_________________________________
_________________________________
FScore is 0.969
_________________________________



# GRU

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'GRU', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Fitting
Epoch 1/4
14039/14039 [==============================] - 13s 902us/step - loss: 0.3635 - acc: 0.8449
Epoch 2/4
14039/14039 [==============================] - 12s 843us/step - loss: 0.1501 - acc: 0.9456
Epoch 3/4
14039/14039 [==============================] - 12s 846us/step - loss: 0.1017 - acc: 0.9640
Epoch 4/4
3510/3510 [==============================] - 1s 247us/step
 0.7 fold 0 precision 0.983 recall 0.957 fscore 0.97
fold 0 average precision 0.996
fold 0 roc auc 0.983
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 13s 894us/step - loss: 0.3947 - acc: 0.8402
Epoch 2/4
14039/14039 [==============================] - 12s 847us/step - loss: 0.1526 - acc: 0.9443
Epoch 3/4
14039/14039 [==============================] - 12s 856us/step - 

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'GRU', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 16s 891us/step - loss: 0.3442 - acc: 0.8601
Epoch 2/4
17549/17549 [==============================] - 15s 856us/step - loss: 0.1335 - acc: 0.9503
Epoch 3/4
17549/17549 [==============================] - 15s 866us/step - loss: 0.0919 - acc: 0.9657
Epoch 4/4
7168/7168 [==============================] - 2s 250us/step
_________________________________
PR-C is 0.997
_________________________________

_________________________________
ROC AUC is 0.983
_________________________________
_________________________________
FScore is 0.971
_________________________________



# CNN

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'CNN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
14039/14039 [==============================] - 8s 593us/step - loss: 0.3769 - acc: 0.8406
Epoch 2/4
14039/14039 [==============================] - 8s 563us/step - loss: 0.1240 - acc: 0.9536
Epoch 3/4
14039/14039 [==============================] - 8s 562us/step - loss: 0.0853 - acc: 0.9695
Epoch 4/4
3510/3510 [==============================] - 1s 179us/step
 0.7 fold 0 precision 0.98 recall 0.963 fscore 0.972
fold 0 average precision 0.997
fold 0 roc auc 0.984
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 8s 582us/step - loss: 0.3828 - acc: 0.8407
Epoch 2/4
14039/14039 [==============================] - 8s 564us/step - loss: 0.1289 - acc: 0.9500
Epoch 3/4
14039/14039 [==============================] - 8s 563us/step - loss: 0.0876 - acc: 0.9691
Epoch 4/4
3510/3510 [==============================] - 1s 163us/step
 0.7 fold 1 precision 0.985 recall 0.963 fscore 0.974
fold 1 average precision 0.997
fold 1 roc auc 0.985
 fold 2
Fitting
Epo

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'CNN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 10s 578us/step - loss: 0.3477 - acc: 0.8498
Epoch 2/4
17549/17549 [==============================] - 10s 565us/step - loss: 0.1138 - acc: 0.9578
Epoch 3/4
17549/17549 [==============================] - 10s 566us/step - loss: 0.0851 - acc: 0.9680
Epoch 4/4
7168/7168 [==============================] - 1s 158us/step
_________________________________
PR-C is 0.997
_________________________________

_________________________________
ROC AUC is 0.985
_________________________________
_________________________________
FScore is 0.971
_________________________________



# BGRU

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.3727 - acc: 0.8400
Epoch 2/4
14039/14039 [==============================] - 24s 2ms/step - loss: 0.1599 - acc: 0.9422
Epoch 3/4
14039/14039 [==============================] - 24s 2ms/step - loss: 0.0979 - acc: 0.9645
Epoch 4/4
3510/3510 [==============================] - 2s 493us/step
 0.7 fold 0 precision 0.983 recall 0.958 fscore 0.97
fold 0 average precision 0.996
fold 0 roc auc 0.981
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.3792 - acc: 0.8372
Epoch 2/4
14039/14039 [==============================] - 24s 2ms/step - loss: 0.1600 - acc: 0.9420
Epoch 3/4
14039/14039 [==============================] - 24s 2ms/step - loss: 0.0989 - acc: 0.9630
Epoch 4/4
3510/3510 [==============================] - 2s 469us/step
 0.7 fold 1 precision 0.978 recall 0.963 fscore 0.97
fold 1 average precision 0.996
fold 1 roc auc 0.983
 fold 2
Fitting
Epoch 1/4


In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False,
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 31s 2ms/step - loss: 0.3574 - acc: 0.8532
Epoch 2/4
17549/17549 [==============================] - 29s 2ms/step - loss: 0.1388 - acc: 0.9487
Epoch 3/4
17549/17549 [==============================] - 30s 2ms/step - loss: 0.0941 - acc: 0.9672
Epoch 4/4
7168/7168 [==============================] - 3s 458us/step
_________________________________
PR-C is 0.996
_________________________________

_________________________________
ROC AUC is 0.981
_________________________________
_________________________________
FScore is 0.963
_________________________________



# BLSTM

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BLSTM', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
14039/14039 [==============================] - 32s 2ms/step - loss: 0.3919 - acc: 0.8486
Epoch 2/4
14039/14039 [==============================] - 30s 2ms/step - loss: 0.1534 - acc: 0.9410
Epoch 3/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.1050 - acc: 0.9615
Epoch 4/4
3510/3510 [==============================] - 2s 666us/step
 0.7 fold 0 precision 0.982 recall 0.952 fscore 0.967
fold 0 average precision 0.996
fold 0 roc auc 0.981
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 32s 2ms/step - loss: 0.4039 - acc: 0.8377
Epoch 2/4
14039/14039 [==============================] - 30s 2ms/step - loss: 0.1579 - acc: 0.9403
Epoch 3/4
14039/14039 [==============================] - 30s 2ms/step - loss: 0.1066 - acc: 0.9624
Epoch 4/4
3510/3510 [==============================] - 2s 623us/step
 0.7 fold 1 precision 0.983 recall 0.955 fscore 0.969
fold 1 average precision 0.996
fold 1 roc auc 0.981
 fold 2
Fitting
Epoch 1/

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BLSTM', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 40s 2ms/step - loss: 0.3485 - acc: 0.8684
Epoch 2/4
17549/17549 [==============================] - 38s 2ms/step - loss: 0.1425 - acc: 0.9459
Epoch 3/4
17549/17549 [==============================] - 38s 2ms/step - loss: 0.1133 - acc: 0.9606
Epoch 4/4
7168/7168 [==============================] - 4s 593us/step
_________________________________
PR-C is 0.996
_________________________________

_________________________________
ROC AUC is 0.982
_________________________________
_________________________________
FScore is 0.962
_________________________________



# BGRU + Max Pooling

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_max', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
14039/14039 [==============================] - 26s 2ms/step - loss: 0.4242 - acc: 0.8385
Epoch 2/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.1549 - acc: 0.9453
Epoch 3/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.1010 - acc: 0.9650
Epoch 4/4
3510/3510 [==============================] - 2s 553us/step
 0.7 fold 0 precision 0.974 recall 0.968 fscore 0.971
fold 0 average precision 0.997
fold 0 roc auc 0.984
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 28s 2ms/step - loss: 0.4185 - acc: 0.8404
Epoch 2/4
14039/14039 [==============================] - 26s 2ms/step - loss: 0.1577 - acc: 0.9405
Epoch 3/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.1034 - acc: 0.9644
Epoch 4/4
3510/3510 [==============================] - 2s 533us/step
 0.7 fold 1 precision 0.978 recall 0.96 fscore 0.969
fold 1 average precision 0.997
fold 1 roc auc 0.983
 fold 2
Fitting
Epoch 1/4

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_max', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 33s 2ms/step - loss: 0.3769 - acc: 0.8491
Epoch 2/4
17549/17549 [==============================] - 32s 2ms/step - loss: 0.1395 - acc: 0.9499
Epoch 3/4
17549/17549 [==============================] - 32s 2ms/step - loss: 0.0963 - acc: 0.9647
Epoch 4/4
7168/7168 [==============================] - 4s 499us/step
_________________________________
PR-C is 0.997
_________________________________

_________________________________
ROC AUC is 0.984
_________________________________
_________________________________
FScore is 0.971
_________________________________



# BGRU + attention

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_att', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
14039/14039 [==============================] - 32s 2ms/step - loss: 0.4239 - acc: 0.8314
Epoch 2/4
14039/14039 [==============================] - 30s 2ms/step - loss: 0.1515 - acc: 0.9415
Epoch 3/4
14039/14039 [==============================] - 33s 2ms/step - loss: 0.0944 - acc: 0.9660
Epoch 4/4
3510/3510 [==============================] - 2s 701us/step
 0.7 fold 0 precision 0.979 recall 0.961 fscore 0.97
fold 0 average precision 0.996
fold 0 roc auc 0.981
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 32s 2ms/step - loss: 0.4128 - acc: 0.8417
Epoch 2/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.1505 - acc: 0.9448
Epoch 3/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.0998 - acc: 0.9648
Epoch 4/4
3510/3510 [==============================] - 2s 672us/step
 0.7 fold 1 precision 0.974 recall 0.967 fscore 0.971
fold 1 average precision 0.997
fold 1 roc auc 0.983
 fold 2
Fitting
Epoch 1/4

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_att', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 42s 2ms/step - loss: 0.3589 - acc: 0.8586
Epoch 2/4
17549/17549 [==============================] - 38s 2ms/step - loss: 0.1408 - acc: 0.9509
Epoch 3/4
17549/17549 [==============================] - 38s 2ms/step - loss: 0.0952 - acc: 0.9660
Epoch 4/4
7168/7168 [==============================] - 5s 636us/step
_________________________________
PR-C is 0.996
_________________________________

_________________________________
ROC AUC is 0.983
_________________________________
_________________________________
FScore is 0.97
_________________________________



# BGRU + Average Pooling

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'BGRU_avg', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False,
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/4
14039/14039 [==============================] - 26s 2ms/step - loss: 0.3881 - acc: 0.8414
Epoch 2/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.1470 - acc: 0.9462
Epoch 3/4
14039/14039 [==============================] - 25s 2ms/step - loss: 0.1099 - acc: 0.9610
Epoch 4/4
3510/3510 [==============================] - 2s 564us/step
 0.7 fold 0 precision 0.989 recall 0.942 fscore 0.965
fold 0 average precision 0.996
fold 0 roc auc 0.982
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 26s 2ms/step - loss: 0.3883 - acc: 0.8457
Epoch 2/4
14039/14039 [==============================] - 24s 2ms/step - loss: 0.1449 - acc: 0.9455
Epoch 3/4
14039/14039 [==============================] - 24s 2ms/step - loss: 0.1059 - acc: 0.9617
Epoch 4/4
3510/3510 [==============================] - 2s 525us/step
 0.7 fold 1 precision 0.974 recall 0.963 fscore 0.969
fold 1 average precision 0.996
fold 1 roc auc 0.982
 fold 2
Fitting
Epoch 1/

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'BGRU_avg', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=maxlen,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 32s 2ms/step - loss: 0.3358 - acc: 0.8676
Epoch 2/4
17549/17549 [==============================] - 31s 2ms/step - loss: 0.1428 - acc: 0.9475
Epoch 3/4
17549/17549 [==============================] - 31s 2ms/step - loss: 0.1007 - acc: 0.9648
Epoch 4/4
7168/7168 [==============================] - 4s 502us/step
_________________________________
PR-C is 0.996
_________________________________

_________________________________
ROC AUC is 0.982
_________________________________
_________________________________
FScore is 0.969
_________________________________



# psHAN

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'psHAN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=33,
                   max_sen_len=17,
                   max_sent_amount=2,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Fitting
Epoch 1/4
14039/14039 [==============================] - 34s 2ms/step - loss: 0.3645 - acc: 0.8490
Epoch 2/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.1339 - acc: 0.9506
Epoch 3/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.1006 - acc: 0.9668
Epoch 4/4
3510/3510 [==============================] - 3s 746us/step
 0.7 fold 0 precision 0.968 recall 0.965 fscore 0.966
fold 0 average precision 0.996
fold 0 roc auc 0.98
 fold 1
Fitting
Epoch 1/4
14039/14039 [==============================] - 33s 2ms/step - loss: 0.3532 - acc: 0.8513
Epoch 2/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.1293 - acc: 0.9530
Epoch 3/4
14039/14039 [==============================] - 31s 2ms/step - loss: 0.0888

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'psHAN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=33,
                   max_sen_len=17,
                   max_sent_amount=2,
                   threshold=threshold)

Fitting
Epoch 1/4
17549/17549 [==============================] - 41s 2ms/step - loss: 0.3170 - acc: 0.8689
Epoch 2/4
17549/17549 [==============================] - 39s 2ms/step - loss: 0.1219 - acc: 0.9543
Epoch 3/4
17549/17549 [==============================] - 38s 2ms/step - loss: 0.0850 - acc: 0.9701
Epoch 4/4
7168/7168 [==============================] - 5s 706us/step
_________________________________
PR-C is 0.996
_________________________________

_________________________________
ROC AUC is 0.981
_________________________________
_________________________________
FScore is 0.969
_________________________________



# HAN

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/twitter_cleaned_sent_punkt.csv') 
train.drop(['count', 'hate_speech', 'offensive_language', 'neither', 'class'], axis=1, inplace=True)
train.tweet.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['tweet', 'mal'])

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42      
X, X_test, y, y_test  = train_test_split(train.tweet, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
models.train_model(X=X, 
                   y=y, 
                   mtype = 'HAN', 
                   cv=True, 
                   nfolds=5, 
                   epochs=4, 
                   cv_models_path=cv_models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=33,
                   max_sen_len=17,
                   max_sent_amount=2,
                   threshold=threshold)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
0
10000
0
Fitting
Epoch 1/4
14058/14058 [==============================] - 35s 2ms/step - loss: 0.3584 - acc: 0.8569
Epoch 2/4
14058/14058 [==============================] - 31s 2ms/step - loss: 0.1411 - acc: 0.9457
Epoch 3/4
14058/14058 [==============================] - 31s 2ms/step - loss: 0.0986 - acc: 0.9624
Epoch 4/4
3515/3515 [==============================] - 3s 819us/step
 0.7 fold 0 precision 0.977 recall 0.955 fscore 0.966
fold 0 average precision 0.994
fold 0 roc auc 0.977
 fold 1
0
10000
0
Fitting
Epoch 1/4
14058/14058 [==============================] - 33s 2ms/step - loss: 0.3585 - acc: 0.8623
Epoch 2/4
14058/14058 [==============================] - 31s 2ms/step - loss: 0.1459 - acc: 0.9457
Epoch 3/4
14058/14058 [==============================] - 31s 2ms/step - loss: 0.0969 - acc: 0.9645
Epoch 4/4
3515/3515 [==============================] - 3s 747us/step
 0.7 fold 1 precision 0.976 recall 0.949 fscore 0.963
fold 1 average precision 0.995
fold 1 roc auc 0.975
 fol

In [0]:
models.train_model(X=X, 
                   y=y, 
                   X_test=X_test, 
                   y_test=y_test, 
                   mtype = 'HAN', 
                   cv=False, 
                   epochs=4, 
                   cv_models_path=models_path, 
                   train=False, 
                   rs=rs,
                   max_features=max_features, 
                   maxlen=33,
                   max_sen_len=17,
                   max_sent_amount=2,
                   threshold=threshold)

0
10000
0
Fitting
Epoch 1/4
17573/17573 [==============================] - 41s 2ms/step - loss: 0.3103 - acc: 0.8716
Epoch 2/4
17573/17573 [==============================] - 39s 2ms/step - loss: 0.1227 - acc: 0.9554
Epoch 3/4
17573/17573 [==============================] - 39s 2ms/step - loss: 0.0918 - acc: 0.9665
Epoch 4/4
7179/7179 [==============================] - 5s 712us/step
_________________________________
PR-C is 0.995
_________________________________

_________________________________
ROC AUC is 0.978
_________________________________
_________________________________
FScore is 0.96
_________________________________

